# Coursera Capstone
## Week 3 Assignment

In [154]:
# imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

1. Using the postal code list from this page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 
reading the website, then loading for beautifulsoup to parse

In [155]:
url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url).text
soup = BeautifulSoup(page,'xml')

2. Using beautifulsoup, extract the cells we want - all rows where the Borough is not assigned - and for rows when the Neighborhood is not assigned, giving it the Borough name. Then loading into a pandas dataframe

In [156]:
table = soup.find('table')
table_rows = table.find_all('tr')
rows = []
for tr in table_rows:
    r = tr.find_all('td')
    if r:
        rows.append(r)
        
data = []
for row in rows:
    a = row[0].text.strip()
    b = row[1].text.strip()
    c = row[2].text.strip()
    if b != 'Not assigned':
        if c == 'Not assigned':
            c = b
        data.append([a, b, c])

df = pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])
#df.head()

3. Now we have a row for every Neighborhood, but we want to use groupby so that each postal code only has one row.

In [162]:
df_2 = df.groupby(by = ['PostalCode','Borough']).agg(lambda x: ','.join(x))
df_2.reset_index(level=['PostalCode','Borough'], inplace=True)
#df_2.head()

4. Using the .shape method to show the # of rows in the dataframe (103 rows):

In [163]:
print(df_2.shape)

(103, 3)


## Part 2

1. Import the provided CSV in order to map coordinates to each Postal Code, and confirm it contains the right information - one row for each postal code (103 rows)

In [164]:
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head()
print(coords.shape)

(103, 3)


2. Join the two tables into an output dataframe providing PostalCode, Borough, list of Neighborhoods, Latitude, an|d Longitudes. Check the head to make sure it's formatted correctly.

In [166]:
joined = df_2.set_index('PostalCode').join(coords.set_index('Postal Code'))
output = joined.reset_index()
output.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


3. Check the shape to make sure we did the join correctly (output should have 103 rows and 5 columns)

In [168]:
print(output.shape)

(103, 5)
